In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score
import models
import process_data
import analysis

In [2]:
#setup
BATCH_SIZE = 200
LEARNING_RATE = 0.0005

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device_count = torch.cuda.device_count()


In [3]:
train_loader, val_loader = process_data.load_data(batch_size=BATCH_SIZE)

In [5]:
cnetmodel = models.get_model("cnet")

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnetmodel.parameters(), lr=LEARNING_RATE)


In [7]:
def eval(model, val_loader):
    model.eval()

    x_pred = []
    x_true = []
    i = 0
    for x, y in val_loader:
        x=x.to(device).reshape(-1, 1, 187).float()
        x_hat = F.softmax(model(x), dim=1)
        x_hat = torch.argmax(x_hat, dim=1)
        x_pred.extend(x_hat.cpu().numpy())
        x_true.extend(y.long().detach().numpy())
    f = f1_score(y_pred=x_pred, y_true=x_true, average='micro')
    acc = accuracy_score(y_pred=x_pred, y_true=x_true)
    return f, acc


In [8]:
def train(model, train_loader, val_loader, n_epochs):
    model.train()
    for epoch in range(n_epochs):
        train_loss = 0
        i=0
        for x, y in train_loader:
            x=x.to(device).reshape(-1, 1, 187).float()
            y = y.to(device).long()
            y_hat = model(x)
            loss = criterion(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        f, acc = eval(model, val_loader)
        print('Epoch: %d \t Validation f: %.2f, acc: %.2f'%(epoch+1, f, acc))

In [10]:
train(cnetmodel, train_loader, val_loader, 25)


Epoch: 1 	 Training Loss: 0.213670
Epoch: 1 	 Validation f: 0.96, acc: 0.96
Epoch: 2 	 Training Loss: 0.125276
Epoch: 2 	 Validation f: 0.97, acc: 0.97


In [ ]:
embeds,y = analysis.get_embeddings(cnetmodel, train_loader)
coordinates = analysis.get_embeddings(embeds)
analysis.plot_tsne(coordinates, y, 5)
